**Project 17: Analysis of Online Reviews of Emergency apps using NLP and Technology Acceptance Models (TAM).**

In this project, we collect online reviews from Apple and Google Play stores from two emergency apps(Emergency Plus , First Aid: American Red Cross), to understand how users react to them.

**Task 1 and Task 2 : Preprocessing**

In [2]:
from google_play_scraper import reviews_all
import pandas as pd
from deep_translator import GoogleTranslator
from app_store_scraper import AppStore
import numpy as np

def PrepareDataForTranslation(reviews_dataframe):
  global valueToFillForEmptyReviews
  valueToFillForEmptyReviews = "No review Comment"
  reviews_dataframe['Review'] = reviews_dataframe['Review'].fillna(valueToFillForEmptyReviews)
  reviews_dataframe['Review'].str.replace('|', '')

  splittedArray =[]
  characterLength =0
  arrayChunk =[]
  for i in range(len( reviews_dataframe['Review'])):
    characterLength = characterLength + len(reviews_dataframe['Review'][i])
                                            
    if characterLength > 3000 :
        splittedArray.append(arrayChunk)
        arrayChunk =[]
        characterLength= len(reviews_dataframe['Review'][i])
        arrayChunk.append(reviews_dataframe['Review'][i])
                             
    else:
      arrayChunk.append(reviews_dataframe['Review'][i])

    if i == len(reviews_dataframe['Review'])-1 :
       splittedArray.append(arrayChunk)
   
  return splittedArray


def TranslateReviewContent(splittedReviewList):
  translatedReviews =[]
  concatednatedReviewString=""
  for i in range(len(splittedReviewList)):
     concatednatedReviewString = "|".join(splittedReviewList[i])
     translatedValue = translator.translate(concatednatedReviewString, dest='en')
     splittedTranslatedContent= translatedValue.split("|")
     translatedReviews = translatedReviews + splittedTranslatedContent
  return translatedReviews


def ExtractAndTranslateGoogleReviews(app):
  googleReviewResult = reviews_all( app['googleId'], 
                       lang= app['googleLanguage'], 
                       country=app['googleCountry'], 
                       )

  googleAppReviewsdf = pd.DataFrame(googleReviewResult)
  googleAppReviewsdf.rename(columns = {'reviewId':'Review Id','userName':'User Name', 'content':'Review', 'score':'Rating', 'at':'Date of Review'}, inplace = True)
  splittedReviewList = PrepareDataForTranslation(googleAppReviewsdf)
  googleAppReviewsdf['Review'] = TranslateReviewContent(splittedReviewList);
  googleAppReviewsdf['Review'].replace([valueToFillForEmptyReviews], '')
  return googleAppReviewsdf

def ExtractAndTranslateAppleReviews(app):
  appleApp = AppStore(app_name=app['appStoreName'], app_id = app['appleAppId'], country=app['appleCountry'])
  appleApp.review(how_many=2000)
  appleAppReviewsdf = pd.DataFrame(appleApp.reviews)
  appleAppReviewsdf.dropna(how='all')
  appleAppReviewsdf.rename(columns = {'userName':'User Name', 'review':'Review', 'rating':'Rating', 'date':'Date of Review'}, inplace = True)
  splittedAppleReviewList = PrepareDataForTranslation(appleAppReviewsdf)
  appleAppReviewsdf['Review'] = TranslateReviewContent(splittedAppleReviewList);
  appleAppReviewsdf['Review'].replace([valueToFillForEmptyReviews], '')
  return appleAppReviewsdf





emergencyApps = [{'appName': 'Emergency Plus','googleId': 'com.threesixtyentertainment.nesn', 'googleLanguage': 'en', 'googleCountry': 'us', 
                                    'appStoreName':'emergency-plus','appleAppId' : '691814685', 'appleCountry': 'au'},
                  {'appName': 'Red Cross First Aid','googleId': 'com.cube.arc.fa', 'googleLanguage': 'en', 'googleCountry': 'us', 
                                    'appStoreName':'first-aid-american-red-cross','appleAppId' : '529160691', 'appleCountry': 'us'}
]

headers = ["Review Id", "User Name" ,"Review","Rating", "Date of Review", "reviewCreatedVersion"] 
translator = GoogleTranslator(source='auto', target='en')

for app in emergencyApps: 
  
  googleAppReviewsdf = ExtractAndTranslateGoogleReviews(app)
  appleAppReviewsdf = ExtractAndTranslateAppleReviews(app)

  combinedReviewsdf = pd.concat([googleAppReviewsdf, appleAppReviewsdf])
  combinedReviewsdf['Date of Review'] = combinedReviewsdf['Date of Review'].dt.date
  combinedReviewsdf.to_csv("./data/Task 1/Result/"+app['appName']+'.csv', index=None, columns = headers, header=True)

  print(app['appName']+".csv file is created sucessfully. Check the location data/Task 1/Result")

C:\Users\Sasini\AppData\Local\Temp\ipykernel_10028\3061472037.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  reviews_dataframe['Review'].str.replace('|', '')
2022-11-06 13:53:28,517 [INFO] Base - Initialised: AppStore('au', 'emergency-plus', 691814685)
2022-11-06 13:53:28,518 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/au/app/emergency-plus/id691814685
2022-11-06 13:53:29,513 [INFO] Base - [id:691814685] Fetched 201 reviews (201 fetched in total)
C:\Users\Sasini\AppData\Local\Temp\ipykernel_10028\3061472037.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  reviews_dataframe['Review'].str.replace('|', '')


Emergency Plus.csv file is created sucessfully. Check the location data/Task 1/Result


IncompleteRead: IncompleteRead(0 bytes read)